Created on Lundi 11 January 2021

Group 5 - Classification
Extraction features syntaxe

@authors : F.B

In [ ]:
import pandas as pd 
import numpy as np
import re
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.metrics import f1_score
from string import punctuation
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os 
os.chdir('/content/drive/MyDrive/G5 Inter-Promo 2021/Données/Input/')

Mounted at /content/drive


In [ ]:
df_analyse = pd.read_json("df_final_clean.json")
df_analyse.drop('art_content', axis=1, inplace=True)

df_train = pd.read_csv("Data_With_Features_Syntax.csv")
df_train = df_train.merge(df_analyse)
df_train.index = df_train.art_id.values

In [ ]:
def clean_keyword(link_keyword : str) -> list:
  ''' This function allows you to clean up keywords
    Input :
      link_keyword : link of keyword
    Output : 
      list_key_word : list of cleaned keywords 
  '''
  df_lexique: pd.DataFrame = pd.read_csv(link_keyword, sep="  ", header=None)
  df_lexique.columns: list = ['key_word']
  list_key_word = df_lexique.key_word.values.tolist()
  list_key_word_net : list = []

  for k in list_key_word :
    k=k.lower()
    k=k.replace(' de ',' ')
    k=k.replace(" d' ",' ')
    k=k.replace("."," ")
    k=k.replace("-"," ")
    k=k.replace(" du "," ")
    list_key_word_net.append(k)

  list_key_word_net = pd.unique(list_key_word_net).tolist()
  return list_key_word_net

link_keyword = "/content/drive/MyDrive/G5 Inter-Promo 2021/Ressources/Lexique_Innovation.txt"
list_key_word = clean_keyword(link_keyword)

In [ ]:
# Counts the number of words
def nb_word(text: str) -> int:
    """Documentation
      Parameters:
        text: text of the article

      Out (if exists):
        nb_word: number of word in  the text
    """
    nb_words: list = []
    nb: int = 0
    # removes special characters
    for p in punctuation:
      text= text.replace(p, ' ')

    # counts the number of words present in the text
    return len(text.split())


In [ ]:
# Count the number of time where the words in the list appear

def count_key_words(text : str) -> int:
    """Documentation
    Parameters:
        text: text of the article

    Out (if exists):
        t : number of key word in  the text
    """
    t : int = 0

    if text != None:
        text = text.lower()
        text = text.split()
        for j in range(len(text)):
            if (text[j] in list_key_word):   #list_key_word: List of word that we will check in the sentences
                t += 1
            else : # For 2-word keywords
              try : # In case j+1 does not exist
                if text[j]+' '+text[j+1] in list_key_word :
                  t += 1
                else : #For 3-word keywords
                  try : # In case j+2 does not exist
                    if text[j]+' '+text[j+1]+' '+text[j+2] in list_key_word :
                      t+=1
                  except :
                    pass
              except :
                pass
    return t

In [ ]:
# Count the number of sentence
def phrases( text : str) -> int:
    """Documentation
    Parameters:
        text: text of the article

    Out (if exists):
        n:  The number of sentence in a article
    """

    n : int = 0
   
    if not isinstance(text, str):
        text: str = str(text)

    if (text != None):
        text = text.replace("..", ".")
        text = text.replace("...", ".")
        text = text.replace("!", ".")
        text = text.replace("!!", ".")
        text = text.replace("!!!", ".")
        text = text.replace("?", ".")
        text = text.replace("??", ".")
        text = text.replace("???", ".")
        text = text.replace("?!", ".")
        text = text.replace("!?", ".")
        n = len(sent_tokenize(text))

    return n

In [ ]:
def create_features(data : pd.DataFrame, text : str, title : str) -> pd.DataFrame:
  ''' This function allows you to create features
    Input :
      data : Dataframe
      text : Article contents column
      title : Article title column
    Output :
      data : Dataframe
  '''
  data["Nb_key_words"] : np.DataFrame = data[text].apply(count_key_words)
  data["Nb_key_words_title"] : np.DataFrame = data[title].apply( count_key_words)
  data["Nb_words"]: np.DataFrame = data[text].apply(nb_word)
  data["Nb_words_title"]: np.DataFrame = data[title].apply(nb_word)
  data["Nb_sentences"]: np.DataFrame = data['art_content'].apply(phrases)
  data["average_word_sentence"]: np.DataFrame = data["Nb_words"] / data["Nb_sentences"]
  data['ratio_key_words']: np.DataFrame  = data['Nb_key_words']/data['Nb_words']
  data['ratio_key_words']: np.DataFrame  = data['ratio_key_words'].fillna(0)
  data['ratio_key_sentences']: np.DataFrame  = data['Nb_key_words']/data['Nb_sentences']
  data['ratio_key_sentences']: np.DataFrame  = data['ratio_key_sentences'].fillna(0)
  data['ratio_key_word_title']: np.DataFrame  = data['Nb_key_words_title'] / data['Nb_words_title']
  data['ratio_key_word_title']: np.DataFrame  = data['ratio_key_word_title'].fillna(0)
  
  return data

In [ ]:
%%time
df_train = create_features(df_train, 'art_content_prepd', 'art_title')

CPU times: user 30.3 s, sys: 0 ns, total: 30.3 s
Wall time: 30.4 s


In [ ]:
df_train.head()

,art_id,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag,Nb_key_words,Nb_key_words_title,Nb_words,Nb_words_title,Nb_sentences,average_word_sentence,ratio_word_title_on_word,exclamation,interrogation,ratio_key_words,ratio_key_sentences,ratio_key_word_title,netloc.com,nb_word_path,content_postive_score,title_postive_score,content_negative_score,title_negative_score,content_polarity_score,title_polarity_score,content_subjectivity_score,title_subjectivity_score,art_content_prepd
1,1,La FNCDG et l’ANDCDG ont publié en septembre l...,"<p style=""text-align: justify;"">La FNCDG et l’...",22 septembre 2020,fr,9ème édition du Panorama de l’emploi territorial,http://fncdg.com/9eme-edition-du-panorama-de-l...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/09/im...,NaN,NaN,0,0,26,7,3,8.666667,0.175000,0,0,0.000000,0.000000,0.000000,['.com'],7,0.00,0.0,0.000,0.0,0.000000,0.0,0.000000,0.0,fncdg andcdg publie septembre eme edition pano...
2,2,Malgré la levée des mesures de confinement le ...,"<p style=""text-align: justify;"">Malgré la levé...",17 mars 2020,fr,ACTUALITÉS FNCDG / COVID19,http://fncdg.com/actualites-covid19/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/03/co...,NaN,NaN,0,0,85,3,4,21.250000,0.025478,0,0,0.000000,0.000000,0.000000,['.com'],2,0.03,0.0,0.000,0.0,0.400000,0.0,1.000000,0.0,malgre levee mesure confinement mai plupart me...
25,25,Quels étaient les objectifs poursuivis par le ...,"<p style=""text-align: justify;""><strong>Quels ...",24 octobre 2019,fr,"Interview de M. Olivier DUSSOPT, Secretaire d’...",http://fncdg.com/interview-de-m-olivier-dussop...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2019/10/in...,NaN,NaN,2,0,740,19,54,13.703704,0.014179,0,4,0.002703,0.037037,0.000000,['.com'],19,0.00,0.0,0.006,0.0,0.026786,0.1,0.261905,0.1,quels etaient objectif poursuivis gouvernement...
27,27,"La journée thématique, qui aura lieu durant le...","<p style=""text-align: justify;""><strong>La jo...",31 mai 2017,fr,Journée Thématique FNCDG « Les services de san...,http://fncdg.com/journee-thematique-fncdg-les-...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/05/pu...,NaN,NaN,0,1,76,13,3,25.333333,0.104839,0,0,0.000000,0.000000,0.076923,['.com'],11,0.00,0.0,0.000,0.0,0.000000,0.0,0.000000,0.0,journee thematique lieu durant salon preventic...
28,28,La 1ère journée thématique en région sur le th...,"<p style=""text-align: justify;"">La 1<sup>ère</...",13 mars 2017,fr,Journée Thématique FNCDG « Vers de nouveaux mo...,http://fncdg.com/journee-thematique-fncdg-vers...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/03/Sa...,NaN,NaN,0,0,114,12,5,22.800000,0.065934,0,0,0.000000,0.000000,0.000000,['.com'],10,0.00,0.0,0.000,0.0,0.000000,0.0,0.000000,0.0,ere journee thematique region theme ver nouvea...


In [ ]:
df_train.to_json('Data_With_Features_Syntax_v1.json')